### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed sentencepiece

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets
import numpy as np
from sklearn.metrics import accuracy_score

### Load data and model

In [5]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")




Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [3]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Tokenize the data

In [4]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [5]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [6]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [7]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
    predicted = model(
        input_ids=batch['input_ids'],
        attention_mask=batch['attention_mask'],
        token_type_ids=batch['token_type_ids']
    )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [8]:
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=200, shuffle=False, collate_fn=transformers.default_data_collator
)

In [9]:
model.to(device='cuda')

print(val_set.shape)

(40430, 8)


In [10]:
batch_logits = torch.empty(0).to(device='cuda')
for batch in val_loader:
    with torch.no_grad():
        pred = model(
            input_ids=batch['input_ids'].to(device='cuda'),
            attention_mask=batch['attention_mask'].to(device='cuda'),
            token_type_ids=batch['token_type_ids'].to(device='cuda')
        )
    batch_logits = torch.cat((batch_logits, pred['logits'].argmax(axis=1)), 0)
print(batch_logits.shape)

torch.Size([40430])


In [11]:
y_val = val_set[:]['label']

In [12]:
batch_logits = batch_logits.cpu().numpy()

In [13]:
accuracy = accuracy_score(y_val, batch_logits)
accuracy

0.9083848627256987

In [14]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (5 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [2]:
from transformers import Trainer, TrainingArguments
import os

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
device

device(type='cuda')

In [4]:
model = transformers.DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-base').to(device)
tokenizer = transformers.DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-base')

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

In [9]:
print(type(qqp_preprocessed['validation']))

<class 'datasets.arrow_dataset.Dataset'>


In [10]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=400, shuffle=False, collate_fn=transformers.default_data_collator
)
print(val_set.shape)

(40430, 8)


In [11]:
batch_logits = torch.empty(0).to(device='cuda')
for batch in val_loader:
    with torch.no_grad():
        pred = model(
            input_ids=batch['input_ids'].to(device='cuda'),
            attention_mask=batch['attention_mask'].to(device='cuda'),
            token_type_ids=batch['token_type_ids'].to(device='cuda')
        )
    batch_logits = torch.cat((batch_logits, pred['logits'].argmax(axis=1)), 0)
print(batch_logits.shape)

torch.Size([40430])


In [12]:
y_val = val_set[:]['label']
batch_logits = batch_logits.cpu().numpy()
accuracy = accuracy_score(y_val, batch_logits)
accuracy

0.6318327974276527

Init accuracy of DeBERTa-v3: ~0.63

In [23]:
torch.cuda.empty_cache()

In [24]:
train_set = qqp_preprocessed['train']
print(train_set.shape)

(363846, 8)


In [25]:
training_args = TrainingArguments(
    output_dir='./qqp_fine_tuned',
    num_train_epochs=3,
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40
)

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=qqp_preprocessed["train"],
    eval_dataset=qqp_preprocessed["validation"]
)

In [27]:
trainer.train()

Step,Training Loss
500,0.379100
1000,0.318800
1500,0.296000
2000,0.281900
2500,0.264500
3000,0.274800
3500,0.263500


KeyboardInterrupt: ignored

In [29]:
torch.cuda.empty_cache()

In [28]:
torch.save(model, "./deberta_v3_qqp_fine_tuned.pt")

In [7]:
model = torch.load("./deberta_v3_qqp_fine_tuned.pt").to(device)

In [8]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=400, shuffle=False, collate_fn=transformers.default_data_collator
)
print(val_set.shape)
batch_logits = torch.empty(0).to(device='cuda')
for batch in val_loader:
    with torch.no_grad():
        pred = model(
            input_ids=batch['input_ids'].to(device='cuda'),
            attention_mask=batch['attention_mask'].to(device='cuda'),
            token_type_ids=batch['token_type_ids'].to(device='cuda')
        )
    batch_logits = torch.cat((batch_logits, pred['logits'].argmax(axis=1)), 0)
print(batch_logits.shape)

(40430, 8)
torch.Size([40430])


In [9]:
y_val = val_set[:]['label']
batch_logits = batch_logits.cpu().numpy()
accuracy = accuracy_score(y_val, batch_logits)
accuracy

0.890848379915904

### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
model = torch.load("./deberta_v3_qqp_fine_tuned.pt").to(device)
tokenizer = transformers.DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-base')

In [ ]:
def preprocess_questions(question1, question2):
    result = tokenizer(
        question1, question2,
        padding='max_length', max_length=128, truncation=True
    )
    return result

In [ ]:
def find_top_duplicates(input_question, dataset):
    preprocessed = [preprocess_questions(input_question, question) for question in dataset]
    results = []
    for input in preprocessed:
        with torch.no_grad():
            predicted = model(
                input_ids=torch.Tensor([input['input_ids']]).long().to(device),
                attention_mask=torch.Tensor([input['attention_mask']]).to(device),
                token_type_ids=torch.Tensor([input['token_type_ids']]).to(device)
            )
        results.append(predicted.logits.cpu().numpy()[0, 1])
    return np.flip(np.argsort(results))[:5], np.flip(np.sort(results))[:5]

In [ ]:
dataset = [
    "What is the process for baking a chocolate cake from scratch?",
    "How does climate change impact marine ecosystems?",
    "Can you share examples of successful time management in a professional setting?",
    "What are the main principles of quantum mechanics?",
    "Can you explain the historical significance of the Renaissance period?",
    "What are the health benefits of regular exercise?",
    "How do plants undergo photosynthesis?",
    "How can I improve my time management skills?",
    "What is the role of mitochondria in cellular respiration?",
    "Can you describe the steps involved in building a wooden bookshelf?",
    "What are the key features of a democratic political system?",
    "How does the human circulatory system function?",
    "What time management tools and apps do you recommend?"
]
input_question = "Can you provide tips for effective time management?"

In [ ]:
ixds, results = find_top_duplicates(input_question, dataset)

In [ ]:
print(f"Top 5 duplicates for '{input_question}' question:\n")
for i in range(len(ixds)):
    print(f"[{i + 1}] {dataset[ixds[i]]} - score: {results[i]}")

Top 5 duplicates for 'Can you provide tips for effective time management?' question:

[1] How can I improve my time management skills? - score: 0.06389035284519196
[2] Can you share examples of successful time management in a professional setting? - score: -0.3601159453392029
[3] What time management tools and apps do you recommend? - score: -1.008211612701416
[4] How does the human circulatory system function? - score: -3.2848055362701416
[5] What are the health benefits of regular exercise? - score: -3.5059802532196045


__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.